In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
from utils.functions import *
# from utils.get_data_from_mt5 import *

%load_ext autoreload
%autoreload 2

In [2]:
## Dates
start_dt = datetime(2024, 1,1) 
start = f'{start_dt.year}-{start_dt.month}-{start_dt.day}'
end_dt = datetime.today()
end = f'{end_dt.year}-{end_dt.month}-{end_dt.day}'

## download stock data
ticker = 'BOVA11'
yf_ticker = 'BOVA11'+'.SA'
underlying =  yf.download(yf_ticker, start=start, end=end)['Adj Close']

rets = underlying.pct_change().dropna(axis=0)
# rets = np.log(rets)

days = 36

vol = annualize_vol(rets, days)
last_price = round(underlying.tail(1).reset_index(drop=True).iloc[0],2)

print(f'Standart Deviation: {vol:.2%} in last {days} days')

[*********************100%%**********************]  1 of 1 completed

Standart Deviation: 4.55% in last 36 days


## Options Data Collection

In [3]:
exp_date_front = '2024-08-16'
            
options_chain_front = get_options_chain(ticker, exp_date_front)
options_chain_front

,Option,Type,E/A,Moneyness,Strike,Distance,Premium,volume
0,BOVAH1,CALL,A,ITM,108.0,-11.31,12.49,189.20
1,BOVAH2,CALL,A,ITM,118.0,-3.10,4.60,5513973.85
2,BOVAH3,CALL,E,ITM,113.0,-7.20,8.73,61383.50
3,BOVAH11,CALL,E,ITM,115.0,-5.56,7.25,882147.64
4,BOVAH80,CALL,A,ITM,80.0,-34.30,41.47,207819.05
...,...,...,...,...,...,...,...,...
155,BOVAT950,PUT,E,OTM,95.0,-21.98,0.05,21581.09
156,BOVAT960,PUT,E,OTM,96.0,-21.16,0.05,2193.43
157,BOVAT970,PUT,E,OTM,97.0,-20.34,0.05,3739.55
158,BOVAT980,PUT,E,OTM,98.0,-19.52,0.06,2457.65


In [4]:
## Collect front month data to calculate expected move in the operation period  
exp_date_front = '2024-08-16'
            
options_chain_front = get_options_chain(ticker, exp_date_front)
options_chain_front['abs_Distance'] = options_chain_front['Distance'].abs() 
options_chain_front[options_chain_front['Moneyness'] == 'ATM'].sort_values('abs_Distance').head(2)

# options_chain_front[options_chain_front['Moneyness'] == 'OTM'].sort_values('abs_Distance').head(14).sort_values('Strike')

,Option,Type,E/A,Moneyness,Strike,Distance,Premium,volume,abs_Distance
21,BOVAH122,CALL,A,ATM,122.0,0.19,1.61,6334394.32,0.19
101,BOVAT122,PUT,E,ATM,122.0,0.19,1.80,5823203.00,0.19


In [5]:
## Collect back month data to get the strikes and premium of the operation
exp_date_back = '2024-09-20'
            
options_chain_back = get_options_chain(ticker, exp_date_back)
options_chain_back['abs_Distance'] = options_chain_back['Distance'].abs() 
options_chain_back[options_chain_back['Moneyness'] == 'ATM'].sort_values('abs_Distance').head(2)

# options_chain_back[options_chain_back['Moneyness'] == 'OTM'].sort_values('abs_Distance').head(14).sort_values('Strike')

,Option,Type,E/A,Moneyness,Strike,Distance,Premium,volume,abs_Distance
21,BOVAI122,CALL,A,ATM,122.0,0.19,3.60,4806188.55,0.19
104,BOVAU122,PUT,E,ATM,122.0,0.19,2.27,3132472.96,0.19


## Calculating ATM Straddle
This is made to define the expected move within the lifetime of the Options

In [6]:
## calculate expected move within operation period
atm_straddle, lower, upper = atm_short_straddle(options_chain_front, last_price)

ATM Straddle: 3.41
Range Expectation using ATM Straddle 118.97 e 125.79


In [7]:
sd = annualize_vol(rets, days)
print(f'Standard Deviation: {sd:.2%} in last {days} days')

Standard Deviation: 4.55% in last 36 days


In [8]:
from statsmodels import robust
def annualize_vol_mad(r, periods_per_year=252):
    return robust.mad(r)*(periods_per_year**0.5)

mad = annualize_vol_mad(rets, days)
print(f'Mean Absolute Deviation: {mad:.2%} in last {days} days')

Mean Absolute Deviation: 4.28% in last 36 days


In [9]:
def mad_sd_ratio(mad, sd):
    ratio = mad / sd
    return ratio, ratio < 0.8

ratio = mad_sd_ratio(mad, sd)
print(f'Razão MAD/SD: {ratio[0]:.2%}')

Razão MAD/SD: 93.94%


In [10]:
mad_approx = mad * 1.25
vol_approx = vol * .8

print(f'SD real: {vol:.2%} -> MAD approximation: {mad_approx:.2%} - diff: {1-(vol / mad_approx):.2%}')
print(f'MAD real: {mad:.2%} -> SD approximation: {vol_approx:.2%} - diff: {1-(mad / vol_approx):.2%}')

SD real: 4.55% -> MAD approximation: 5.35% - diff: 14.84%
MAD real: 4.28% -> SD approximation: 3.64% - diff: -17.43%


In [11]:
f'{vol_based_short(last_price, vol, 1)}'
f'{vol_based_short(last_price, mad, 1)}'

Range Expectation using ATM Straddle 116.81 e 127.95
Range Expectation using ATM Straddle 117.14 e 127.62


'(117.14497925264334, 127.61502074735665)'

In [12]:
(atm_straddle / last_price * 100) * 1.25

3.4830037587841156

In [13]:
(atm_straddle / last_price * 100)

2.7864030070272925

In [14]:
## ATM straddle as Volatility approximation
# .153 == implied vol
straddle_aproxx = 0.8 * last_price * .131 * np.sqrt(days / 252)
atm_straddle, straddle_aproxx, (straddle_aproxx/atm_straddle)

(3.41, 4.847554623279895, 1.4215702707565674)

## Selecting Options to construct Iron Condor

In [15]:
# Define the strikes using front month ATM straddle as a proxy for expected move
wing_width = 3 
long_put_strike, short_put_strike, short_call_strike, long_call_strike = select_iron_condor_strikes(options_chain_front, last_price, wing_width=wing_width)

# long_put, short_put, short_call, long_call

ATM Straddle: 3.41
Range Expectation using ATM Straddle 118.97 e 125.79


In [16]:
## Collecting the premiuns from the options_chain_back
long_put = options_chain_back[(options_chain_back['Strike'] == long_put_strike) & (options_chain_back['Type'] == 'PUT')]
short_put = options_chain_back[(options_chain_back['Strike'] == short_put_strike) & (options_chain_back['Type'] == 'PUT')]
short_call = options_chain_back[(options_chain_back['Strike'] == short_call_strike) & (options_chain_back['Type'] == 'CALL')]
long_call = options_chain_back[(options_chain_back['Strike'] == long_call_strike) & (options_chain_back['Type'] == 'CALL')] 

list_df = [long_put, short_put, short_call, long_call] 

for df in list_df:
    print(f'{df['Option'].iloc[0]}, {df['Strike'].iloc[0]}, {df['Premium'].iloc[0]}')

BOVAU116, 116.0, 1.05
BOVAU119, 119.0, 1.52
BOVAI126, 126.0, 1.49
BOVAI129, 129.0, 0.7


In [17]:
long_put_premium = long_put['Premium'].iloc[0] 
short_put_premium = short_put['Premium'].iloc[0] 
short_call_premium = short_call['Premium'].iloc[0] 
long_call_premium = long_call['Premium'].iloc[0]

## print premium 
print(f'long_put: {long_put_strike:.2f}, {long_put_premium:.2f}')
print(f'short_put: {short_put_strike:.2f}, {short_put_premium:.2f}')
print(f'short_call: {short_call_strike:.2f}, {short_call_premium:.2f}')
print(f'long_call: {long_call_strike:.2f}, {long_call_premium:.2f}')

long_put: 116.00, 1.05
short_put: 119.00, 1.52
short_call: 126.00, 1.49
long_call: 129.00, 0.70


In [18]:
## Calculate how much each leg (puts vs calls) contribute to total credit 
put_spread =  short_put_premium - long_put_premium 
call_spread =  short_call_premium - long_call_premium
print(f'Call Spread: {call_spread:.2f}\nPut Spread: {put_spread:.2f}')
call_spread_pct = call_spread / (call_spread+put_spread)
print(f'Call side pct: {call_spread_pct:.2%}')

## if call_spread_pct > .70 of total credit
## unless there is some bullish bias, consider only doing the call bear spread
if call_spread_pct > .70:
    print(f'Consider only doing the call bear spread')
    c = call_spread/(wing_width-call_spread)
    print(f'Call side managed ROIC(net){c * .65:.2%}') 
else:    
    print(f'both sides have good premium')

Call Spread: 0.79
Put Spread: 0.47
Call side pct: 62.70%
both sides have good premium


In [19]:
## Calculates total money risked, legs lenght, total credit, managed Tp and managed ROIC
max_loss, gain_range, credit_received, profit, roc_cost, leg_width = iron_condor(
    options_chain_back,
    long_put_strike,
    short_put_strike,
    short_call_strike,
    long_call_strike
        )

Position Risk: 17.40
Gain Range: 7.00
Credit Received/Max Profit: $12.60
Managed Take Profit: $6.55
Managed ROIC (net): 37.66%


## Expected Move and Kelly Criterion

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import math

from utils.bs_funcs import *

%load_ext autoreload
%autoreload 2

In [ ]:
df = yf.download('BOVA11.SA', period='10Y')
rets = df['Close'].pct_change().dropna()

[*********************100%%**********************]  1 of 1 completed


In [ ]:
mu = annualize_rets(rets[:-252])
print(f'Average Annual Returns: {mu:.2%}')

Average Annual Returns: 9.13%


In [ ]:
S = 124.04  # Preço Ação
K = 124  # Strike 
dte = 36 # dia ate venciemnto 
T = dte/252   # ano até vencmento
r = 0.1025 # taxa livre de risco
market_price = 3.23  # Market price of the option

In [ ]:
rv = realized_vol(df['Close'][-dte:-1], 365)
iv = implied_vol(S, K, T, r, market_price)
yz_vol = yz_volatility(df, dte)

# print(f'YZ vol: {yz_vol* (dte/365):.2%}') 
print(f"Realized ann Volatility: {rv:.2%}")
print(f"Implied ann Volatility: {iv:.2%}")
# print(f"Realized dte Volatility: {rv * (dte/365):.2%}")
# print(f"Implied dte Volatility: {iv * (dte/365):.2%}")

Realized ann Volatility: 12.13%
Implied ann Volatility: 11.78%


In [ ]:
sigma = rv

In [ ]:
# expected_move = mad_straddle(S, sigma, dte)
expected_move = straddle(S, sigma, dte)

ceil = math.ceil(K + expected_move)
floor = math.floor(K - expected_move)


print(f'Expected Move: {expected_move:.2f}')
print(f'Expected lower range: {ceil:.2f}')
print(f'Expected upper range: {floor:.2f}')
print(f'Expected Move / Spot: {expected_move / S:.2%}') 

Expected Move: 5.69
Expected lower range: 130.00
Expected upper range: 118.00
Expected Move / Spot: 4.59%


In [ ]:
# def iv_atm_straddle(straddle, S, T):
#     time_years = T / 365
#     return straddle / (S * (time_years ** 0.5))

# iv_atm_straddle(expected_move, S, dte) 

def iv_dte(annual_vol, dte): 
    return annual_vol * np.sqrt(dte/365)
       
yz_dte_vol = iv_dte(sigma, dte)
print(f'Implict Volatility until DTE:{yz_dte_vol:.2%}')
print(f'Implict Move until DTE:{yz_dte_vol*S:.2F}')

Implict Volatility until DTE:3.81%
Implict Move until DTE:4.73


In [ ]:
straddle_price = 4.9
mad_straddle_ap = (straddle_price / S) * 1.25
print(f"{mad_straddle_ap:.2%}")

4.94%


In [20]:
per_years = dte/252
n_scenarios=10000
steps_per_year=252
 
paths = gbm(per_years, n_scenarios, mu, sigma, steps_per_year, K)[:21]
win_pct, loss_above, loss_bellow = plot_paths(paths, ceiling=ceil+1, floor=floor+1)

NameError: name 'dte' is not defined

In [ ]:

# Calculate the recommended fraction of credit to invest (half Kelly)
recommended_fraction = kelly_criterion(win_pct, 0.25, bet_factor=.5, portfolio_size=110)
print(f"Recommended fraction of portfolio for this trade: {recommended_fraction[0]:.2%}")
print(f"Max risk for this trade: R$ {recommended_fraction[1]:.2f}")

Recommended fraction of portfolio for this trade: 9.35%
Max risk for this trade: R$ 10.29
